In [106]:
import pandas as pd                 #import libraries
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.stats import ttest_ind
df = pd.read_csv('query_results.csv')    #reading csv file

In [107]:
print(df)   #print dataframe

            id country gender group     join_dt device   dt  spent
0      1000000     CAN      M     B  2023-01-28      I  NaN    NaN
1      1000001     BRA      M     A  2023-01-27      A  NaN    NaN
2      1000002     FRA      M     A  2023-02-01      A  NaN    NaN
3      1000003     BRA      M     B  2023-01-25      I  NaN    NaN
4      1000004     DEU      F     A  2023-02-04      A  NaN    NaN
...        ...     ...    ...   ...         ...    ...  ...    ...
49077  1049995     BRA      F     B  2023-02-03      A  NaN    NaN
49078  1049996     USA      F     A  2023-01-29      A  NaN    NaN
49079  1049997     BRA      M     B  2023-02-03      A  NaN    NaN
49080  1049998     CAN      M     B  2023-02-03      I  NaN    NaN
49081  1049999     GBR      M     B  2023-01-29      I  NaN    NaN

[49082 rows x 8 columns]


In [108]:
df['spent'].fillna(0, inplace=True)       #replacing null values with 0 in spent column

In [109]:
df_updated = df.pivot_table(index = ['id','group'], values = 'spent', aggfunc = 'sum').reset_index()  #use pivot table to group the id's and groups
print(df_updated)

            id group  spent
0      1000000     B    0.0
1      1000001     A    0.0
2      1000002     A    0.0
3      1000003     B    0.0
4      1000004     A    0.0
...        ...   ...    ...
48938  1049995     B    0.0
48939  1049996     A    0.0
48940  1049997     B    0.0
48941  1049998     B    0.0
48942  1049999     B    0.0

[48943 rows x 3 columns]


In [110]:
filtered_df_a = df_updated.query("group =='A'")     #filter dataframe for group A
print(filtered_df_a)

            id group  spent
1      1000001     A    0.0
2      1000002     A    0.0
4      1000004     A    0.0
7      1000007     A    0.0
8      1000008     A    0.0
...        ...   ...    ...
48930  1049987     A    0.0
48932  1049989     A    0.0
48936  1049993     A    0.0
48937  1049994     A    0.0
48939  1049996     A    0.0

[24343 rows x 3 columns]


In [111]:
filtered_df_b = df_updated.query("group =='B'")       #filter dataframe for group B
print(filtered_df_b)

            id group  spent
0      1000000     B    0.0
3      1000003     B    0.0
5      1000005     B    0.0
6      1000006     B    0.0
10     1000010     B    0.0
...        ...   ...    ...
48935  1049992     B    0.0
48938  1049995     B    0.0
48940  1049997     B    0.0
48941  1049998     B    0.0
48942  1049999     B    0.0

[24600 rows x 3 columns]


In [112]:
avg_spent_a = filtered_df_a['spent'].mean()    #average amount spent per user group A
print(avg_spent_a)

3.374518467928841


In [113]:
avg_spent_b = filtered_df_b['spent'].mean()     #average amount spent per user group B
print(avg_spent_b)

3.390866945885783


In [114]:
std_dev_a = filtered_df_a['spent'].std()       #std dev of amount spent per user group A
print(std_dev_a)

25.93639055722059


In [115]:
std_dev_b = filtered_df_b['spent'].std()       #std dev of amount spent per user group B
print(std_dev_b)

25.414109599225664


In [116]:
t=1.960              #critical value at 0.05 significance level

margin_of_error_a = t*std_dev_a/math.sqrt(len(filtered_df_a))       #margin of error for average amount spent per user for Group A 
print(margin_of_error_a)

0.3258206149989957


In [117]:
upper_bound_avg_a = avg_spent_a + margin_of_error_a
print(upper_bound_avg_a)

3.7003390829278366


In [118]:
lower_bound_avg_a = avg_spent_a - margin_of_error_a
print(lower_bound_avg_a)

3.0486978529298456


In [119]:
margin_of_error_b = t*std_dev_b/math.sqrt(len(filtered_df_b))       #margin of error for average amount spent per user for group B 
print(margin_of_error_b)

0.3175875114886324


In [120]:
upper_bound_avg_b = avg_spent_b + margin_of_error_b
print(upper_bound_avg_b)

3.7084544573744154


In [121]:
lower_bound_avg_b = avg_spent_b - margin_of_error_b
print(lower_bound_avg_b)

3.0732794343971506


In [122]:
result = ttest_ind(filtered_df_b['spent'],filtered_df_a['spent'])      # t test for average amount spent per user in both groups
print(result)

Ttest_indResult(statistic=0.07043243220818624, pvalue=0.9438497659410893)


In [123]:
se = np.sqrt((std_dev_a**2/len(filtered_df_a))+(std_dev_b**2/len(filtered_df_b)))           #std error for difference in average
print(se)

0.23214055875284387


In [124]:
tstat = (avg_spent_b-avg_spent_a)/se
print(tstat)

0.07042490999751483


In [125]:
diff_avg_spent = avg_spent_b-avg_spent_a          #difference in average
upper_bound_diff = diff_avg_spent + t * se
lower_bound_diff = diff_avg_spent -t * se
print(lower_bound_diff, upper_bound_diff)

-0.43864701719863214 0.4713439731125158


In [126]:
conversion_a = len(filtered_df_a[filtered_df_a['spent']!=0])/len(filtered_df_a)       #conversion rate of group A
print(conversion_a)

0.03923099042845993


In [127]:
conversion_b = len(filtered_df_b[filtered_df_b['spent']!=0])/len(filtered_df_b)      #conversion rate of group B
print(conversion_b)

0.04630081300813008


In [152]:
filtered_df_a.loc[filtered_df_a['spent'] !=0, 'converted_a'] = 1      #creating a new column in the dataset to get who have be converted and not
filtered_df_a.loc[filtered_df_a['spent'] ==0, 'converted_a'] = 0

print(filtered_df_a.reset_index(drop=True))

            id group  spent  converted_a
0      1000001     A    0.0          0.0
1      1000002     A    0.0          0.0
2      1000004     A    0.0          0.0
3      1000007     A    0.0          0.0
4      1000008     A    0.0          0.0
...        ...   ...    ...          ...
24338  1049987     A    0.0          0.0
24339  1049989     A    0.0          0.0
24340  1049993     A    0.0          0.0
24341  1049994     A    0.0          0.0
24342  1049996     A    0.0          0.0

[24343 rows x 4 columns]


In [153]:
filtered_df_b.loc[filtered_df_b['spent'] !=0, 'converted_b'] = 1         #creating a new column in the dataset to get who have be converted and not
filtered_df_b.loc[filtered_df_b['spent'] ==0, 'converted_b'] = 0

print(filtered_df_b.reset_index(drop=True))

            id group  spent  converted_b
0      1000000     B    0.0          0.0
1      1000003     B    0.0          0.0
2      1000005     B    0.0          0.0
3      1000006     B    0.0          0.0
4      1000010     B    0.0          0.0
...        ...   ...    ...          ...
24595  1049992     B    0.0          0.0
24596  1049995     B    0.0          0.0
24597  1049997     B    0.0          0.0
24598  1049998     B    0.0          0.0
24599  1049999     B    0.0          0.0

[24600 rows x 4 columns]


In [130]:
std_converted_a = filtered_df_a['converted_a'].std()    #std dev of conversion of group A
print(std_converted_a)

0.19414805754879985


In [131]:
std_converted_b = filtered_df_b['converted_b'].std()    #std dev of conversion of group B
print(std_converted_b)

0.2101400551961162


In [132]:
se_conv_a = np.sqrt(conversion_a*(1-conversion_a)/len(filtered_df_a))
print(se_conv_a)

0.001244334320502947


In [133]:
se_conv_b = np.sqrt(conversion_b*(1-conversion_b)/len(filtered_df_b))
print(se_conv_b)

0.0013397768246823593


In [134]:
z=1.96
moe_conv_a = z * se_conv_a
print(moe_conv_a)

0.002438895268185776


In [135]:
upper_bound_conversion_a = conversion_a + moe_conv_a
print(upper_bound_conversion_a)

0.0416698856966457


In [136]:
lower_bound_conversion_a = conversion_a - moe_conv_a
print(lower_bound_conversion_a)

0.03679209516027415


In [137]:
moe_conv_b = z * se_conv_b
print(moe_conv_b)

0.002625962576377424


In [138]:
upper_bound_conversion_b = conversion_b + moe_conv_b
print(upper_bound_conversion_b)

0.048926775584507504


In [139]:
lower_bound_conversion_b = conversion_b - moe_conv_b
print(lower_bound_conversion_b)

0.04367485043175266


In [140]:
#pooled proportion
pooled_prop= (len(filtered_df_a[filtered_df_a['spent']!=0])+len(filtered_df_b[filtered_df_b['spent']!=0]))/(len(filtered_df_a)+len(filtered_df_b))
print(pooled_prop)

0.04278446355965102


In [141]:
se_pooled_prop = math.sqrt((pooled_prop*(1-pooled_prop)/len(filtered_df_a))+(pooled_prop*(1-pooled_prop)/len(filtered_df_b)))
print(se_pooled_prop)

0.001829526081285274


In [142]:
z_pooled_stat = (conversion_a -conversion_b)/se_pooled_prop       #z statistic for pooled proportion
print(z_pooled_stat)

-3.864291770414927


In [143]:
from scipy.stats import norm
p_value = 2*(1-norm.cdf(abs(z_pooled_stat)))
print("p-value: {:.6f}".format(p_value))

p-value: 0.000111


In [144]:
result_conv= ttest_ind(filtered_df_b['converted_b'], filtered_df_a['converted_a'])
print(result_conv)

Ttest_indResult(statistic=3.864802444795058, pvalue=0.00011132252128659755)


In [145]:
se_diff_conv = math.sqrt(se_conv_a**2 +se_conv_b**2)     #std error of the conversion rate for both groups
print(se_diff_conv)

0.0018284884033368863


In [146]:
point_est_diff_conv = conversion_b - conversion_a    #difference in conversion
print(point_est_diff_conv)

0.0070698225796701555


In [147]:
z1=1.96      #critical value z
moe_diff_conv = z1 * se_diff_conv      #margin of error for difference in conversion
print(moe_diff_conv)

0.003583837270540297


In [148]:
upper_bound_diff_conv = point_est_diff_conv + moe_diff_conv
lower_bound_diff_conv = point_est_diff_conv - moe_diff_conv
print(lower_bound_diff_conv, upper_bound_diff_conv)

0.0034859853091298585 0.010653659850210453


In [149]:
dict_of_lists = { 'groups': ['group A', 'group B'],
                   'avg_spent_per_user':[avg_spent_a, avg_spent_b],
                   'std_dev_avg' : [std_dev_a, std_dev_b],
                  'total_users' :[len(filtered_df_a), len(filtered_df_b)],
                   'conversion_rate': [conversion_a, conversion_b],
                   'std_converted':[std_converted_a, std_converted_b],
                   'upper_bound_avg': [upper_bound_avg_a,upper_bound_avg_b],
                   'lower_bound_avg':[lower_bound_avg_a,lower_bound_avg_b],
                'upper_bound_conv':[upper_bound_conversion_a,upper_bound_conversion_b],
                'lower_bound_cov':[lower_bound_conversion_a,lower_bound_conversion_b]}
final_table = pd.DataFrame(dict_of_lists)
final_table

,groups,avg_spent_per_user,std_dev_avg,total_users,conversion_rate,std_converted,upper_bound_avg,lower_bound_avg,upper_bound_conv,lower_bound_cov
0,group A,3.374518,25.936391,24343,0.039231,0.194148,3.700339,3.048698,0.041670,0.036792
1,group B,3.390867,25.414110,24600,0.046301,0.210140,3.708454,3.073279,0.048927,0.043675


In [150]:
final_table.to_csv('dict_of_lists.csv', index=False)


In [151]:
df.to_csv('dataframe.csv', index=False)